# Introduction
Welcome to the Pygame Modelling Workshop. You can find the track and tasks in this notebook. We assume that you have `python3.7+` with `pip` installed and you have installed the provided package `pygmodw22` with all it's dependencies in a virtual environment from which you are running this notebook. In case you have not yet done the preparatory steps, please follow the instructions [in the README file](https://github.com/mezdahun/PygameModelling22#1-prerequisites)

## Goal of the Workshop
During this workshop you will get a hands-on introduction and demonstration on modelling a typical multi-agent system using an open-source game engine (pygame). 

The main goals of this workshop is to:
1. Give an intorduction into the used flocking model
2. Give an introduction on how agent-based simulations are usually implemented and how they are implemented in the current pygame-based approach
3. Discover the provided system by real-time hands-on experimentation
4. (Optionally) look deeper in the code

# Part I.: The Three-Zone-Model

Here, we will explore an agent-based model for simulating collective dynamics of animals (e.g. fish schools) through the provided package and small tasks. Feel free to provide your answers in this notebook. Our main task is the exploration of collective dynamics in a 2D model, related to the Three-Zone-Model (Two-Zone-Models) [by Couzin et al.](https://www.sciencedirect.com/science/article/pii/S0022519302930651), where the interaction between individuals is governed by three basic interactions: long-range attraction, short-ranged repulsion and alignment. 

## Brief Introduction
### Fundamental model

The code solves a set of (stochastic) differential equation describing a set of $N$ interacting agents ($i= 1,\dots, N$). The dynamics of each agent (in 2d) is described by the following equations of motion:

$$ \frac{d \vec{r}_i}{dt}=\vec{v}_i(t) $$
$$ \vec{v}_i(t) = {s_i\cos(\varphi_i(t)) \choose s_i\sin(\varphi_i(t)) } $$
$$ \frac{d \varphi_i}{dt} = \frac{1}{s_i}\left( F_{i,\varphi} + \eta_{i,\varphi} \right) $$


Here $\vec{r}_i$, $\vec{v}_i$ are the Cartesian position and velocity  vectors of the focal agent, wth $s_i$ being the (constant) speed of agent $i$. Furthermore, $\eta_{i,\varphi}$ represents Gaussian white noise introducing randomness in the motion of individuals, and $\vec{F}_{i,\varphi}$ is the projections of the total social force inducing a turning behavior.
$$ F_{i,\varphi}=\vec{F}_i \cdot \vec{u}_{\varphi,i} = \vec{F}_i {- s_i\sin\varphi_i \choose s_i\cos\varphi_i } $$


The total effective social force is a sum of three components:
$$ \vec{F}_i=\vec{F}_{i,rep}+\vec{F}_{i,alg}+\vec{F}_{i,att} $$


**Attraction:
$$\vec{F}_{i,att}=\sum_{j \in Neigh} +\mu_{att}S_{att}({r}_{ji}) \hat{r}_{ji} $$
Repulsion:
$$\vec{F}_{i,rep}=\sum_{j \in Neigh} -\mu_{rep}S_{rep}({r}_{ji}) \hat{r}_{ji}$$
Alignment:**
$$\vec{F}_{i,alg}=\sum_{j \in Neigh} \mu_{alg}S_{alg}({r}_{ji}) (\vec{v}_j-\vec{v}_i)$$
with $\hat r = \vec{r}/|r|$.

The strength of the different interactions is set by a constant $\mu_X$ and a sigmoid function of distance, which goes from 1 to 0, with the transition point at $r_{X,0}$ and steepness $a_{X}$:
$$ S_X(r)=\frac{1}{2}\left(\tanh(-a(r-r_{X,0})+1\right) $$

<img src="data/images/scheme_ranges.png" width='800'>

**Figure2.:** Local interaction forces around an agent

<img src="data/images/int_ranges.png">

**Figure1.:** Example of the 3 interaction zones around a focal agent

# Part II.: Agent-based Simulations in pygame

[pygame](https://www.pygame.org/wiki/about) is a free, portable, highly optimized (in terms of CPU usage and multi-threading) game engine available via the main python package manager `pip`. `pygame` was originally designed to create videogames in an easy-to understand and open-source way. Many aspects and challenges of creating videogames, on the other hand, also apply to scientific agent-based simulations (e.g. parallel computing, speed optimization, highly object-oriented design). Next to providing an object-oriented and optimized framework to implement and simulate mathematical models, `pygame` also provides a unique way of interactive experimentation with the studied system via user interaction events (button press, cursor movement, etc.).

## Structure of the provided code base

To make our experimentation faster we have provided a code base including an implementation of the described Three-zone-model as a custom made python package called `pygmodw22` (**Pyg**ame **Mod**elling **W**orkshop 20**22**). The package includes a `setup.py` file that allows it to be installed via `pip` including all dependencies and version restrictions for the package. Inside the folder package `pygmodw22` you have 3 files.

1. **agent.py**: Includes the implementation of moving agents of the model as a single `Agent` class and the rules/methods of how a these agents' behavior is updated in each timestep (`update` method). Other than functionality, the `Agent` class also describes the appearance of the agents (color, radius, etc. `draw` method). Defining and drawing the agents is surpisingly easy compared to other animation methods (e.g. matplotlib animation library). This is one of the great advantages of `pygame`. Each agent class instance will include their own parameters (radius, zone parameters, etc.) hence `pygame` also makes heterogeneous system implementations easy compared to vectorized or non-object-oriented solutions.

2. **sims.py**: In this file we defined the `Simulation` class that defines the arena (in which the agents move), possible user interactions, as well as the main simulation loop. This loop will have $T$ iterations and in each timestep it will call the agents' `update` and `draw` method (among other supplementary methods).

3. **support.py**: Includes all supllementary and mathematical methods needed for the update process. These are, for example, calculation of euclidian distance, implementation of the sigmoid function, etc.

To create an example simulation, see the code below:


## Example 1.: Understanding the Simulation and Agent classes

In [3]:
# first we import the Simulation class from the code base
from pygmodw22.sims import Simulation

In [4]:
# now we can create a Simulation class instance as follows
simulation = Simulation(N=10,  # Number of agents in the environment
                        T=250,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Starting main simulation loop!
2022-06-07_11-11-00.195801 Total simulation time:  10.280757


## Interactions
Pygame provides us a unique way to visualize and interact with complex systems real-time while the simulation/game is running. We implemented a few interactions that you can use throughout the workshop to better understand the system.

**Keystrokes:**

- `f`aster: increase framerate (if your system allows)
- `s`lower: decrease framerate
- `d`efault: default framerate (25fps)
- `c`olor: turn on/off coloration according to agent orientation and velocity
- `z`ones: turn on/off showing the 3 zones around the agent
- `space`: pause/continue simulation

**Cursor Events**:

- **move**: You can drag and drop agents around by clicking and holding them. This allows you to perturb the system without any need of coding.
- **rotate**: You can rotate the agents by first grabbing them and then using your mouse wheel (or scrolling event).



## Task 1: Exploring Interactions

Use the example code snippet below to simulate a swarm with a given parameter set corresponding the 3 interaction zones. Modify the code to increase the simulation time to 2500 timesteps and decrease the number of agents to 5. 

Due to local interactions the 5 agents will shortly align their movement into the same direction. Pause the experiment with `space`. Rotate one of the agents by 180 degrees from it's original orientation by moving your cursor on top of it and scrolling your mouse wheel. Continue the simulation with `space`. 

1. How does the system react to your perturbation? You can also turn on coloration of the agents by their orientation (with `c`) to get an even better insight.
3. Now instead of turning an agent move it further away from the group with your cursor. What do you see?


In [5]:
# first we import the Simulation class from the code base
from pygmodw22.sims import Simulation
# now we can create a Simulation class instance as follows
simulation = Simulation(N=3,  # Number of agents in the environment
                        T=2500,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Starting main simulation loop!
2022-06-07_09-34-45.141477 Total simulation time:  55.873393


## Task 2: Exploring Zones of Agents

Use the code example code snippet below. Modify it to increase the simulation time to 2500 timesteps and decrease the number of agents to only a pair of 2. 

Turn on the visualization of the local interaction zones by pressing `z`. The long-range attraction zone is denoted with a green, the intermediate alignment zone with a yellow and the short-range repulsion with a red circle around the agents. Without pausing the simulation hold one of the agents still with your cursor.  

1. How does the other agent react? Is there anything surprising or different than what you expected?
2. Why? How can you explain what you see with the effect of the 3 zones (attraction/alignment/repulsion)?
3. What happens if you also start rotating the agent at the same time you hold it? Which zone is responsible for the temporary change in the behavior? 

In [5]:
# first we import the Simulation class from the code base
from pygmodw22.sims import Simulation
# now we can create a Simulation class instance as follows
simulation = Simulation(N=2,  # Number of agents in the environment
                        T=2500,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# Let's start the main simulation loop by calling the start method of the created simulation
simulation.start()

Running simulation start method!
Starting main simulation loop!
Bye bye!


SystemExit: 

/home/david/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Example 2: Flocking Parameters

As we have seen in the introduction to the zonal model each agent has 3 zones of local interaction parametrized by their interaction ranges, stepness parameters and interaction strengths.

In the next section we will see how systematically changing these parameters will influence the collective behavior of the system.

When we create a `simulation` object it has N `Agent` type objects in it's `agents` class-attribute which is the list of agents in the environment. They by default have some flocking parameters such as interaction strengths and ranges. To override these default values and set our desired parameters you can do as in the following code example (here we only changed the speed of the agents compared to the default values but we listed other parameters of the agents as well): 

In [10]:
from pygmodw22.sims import Simulation

# We create a Simulation class instance with it's default agents as before
simulation = Simulation(N=20,  # Number of agents in the environment
                        T=2500,  # Simulation timesteps
                        width=500,  # Arena width in pixels
                        height=500,  # Arena height in pixels
                        agent_radius=10)  # Agent radius in pixels

# we loop through all the agents of the created simulation
print("Setting parameters for agent", end = ' ')
for agent in simulation.agents:
    print(f"{agent.id}", end = ', ')
    
    # changing angular noise (sigma)
    agent.noise_sig = 0.1
    
    # changing their default flocking parameters
    agent.s_att = 0.02  # attraction strength (AU)
    agent.s_rep = 5  # repulsion strength (AU)
    agent.s_alg = 10  # alignment strength (AU)

    agent.r_att = 200  # attraction range (px)
    agent.r_rep = 50  # repulsion range (px)
    agent.r_alg = 100  # alignment range (px)
    
    agent.steepness_att = -0.5  # steepness in attraction force calculation (sigmoid)
    agent.steepness_rep = -0.5  # steepness in repulsion force calculation (sigmoid)
    agent.steepness_alg = -0.5  # steepness in alignment force calculation (sigmoid)
    
    # changing maximum velocity and simulation timesteps
    agent.v_max = 2
    agent.dt = 0.05
    
    
# Now we can start the simulation with the changed agents
simulation.start()

Setting parameters for agent 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, Running simulation start method!
Starting main simulation loop!
2022-06-07_11-29-28.048067 Total simulation time:  116.040628


## Task 3: Exploring Flocking Parameters

Use the example code snippet above and modify it to answer the following questions. 

1. **Individual Dynamics**: Turn off all the interaction forces. Perform simulations with different angular noise values (noise_sig) and explore the behavior of the agents (you can try: 0.1, 1, 3). For the next tasks fix this parameter to 0.1.
2. **Obstacle Avoidance**:
    - a. Re-introduce a strong local repulsion interaction (`s_rep = 5`) and by that implement obstacle avoidance in a group of 10 agents. Set the repulsion steepness to -1.
    - b. Test the repulsion beahvior with different repulsion ranges (20, 50, 150). Feel free to move around the agents and see what happens when you move them in each others' repulsion zone.
    - c. **Hint:** You can visualize the zones around the agents with `z`. Only those zones will be shown for which the corresponding interaction strength is larger than zero. 
    - d. Explore the effect of the repulsion steepness (-0.1, -0.5, -1). When does repulsion take place with low or high steepness parameter? 
    - e. Do you think this obstacle avoidance will always avoid agents from collision?
    - f. For the upcoming experimentation fix repulsion strength to 5, steepness to -0.5 and range to 50
 
3. **Attraction-Repulsion**
    - a. Re-intorduce attraction and explore the beahavior with different attraction strenths 0.05, 0.5, 1, 5.
    - b. What happens for attraction strength of 0.05 and 5? What would you consider a realistic attraction strength for a mosquito swarm?
    - c. For the next experiment fix the attraction strength to 0.02, attraction range to 200 and steepness to -0.5.
    
4. **Movement Coordination**
    - a. What do you think you will observe when you introduce an intermediate zone where agents align to their neighbors?
    - b. Re-introduce the alignment zone by setting a strong alignment strength of 5. Is your observation matching with what you expected? What happens if you increase the attraction strength to 2?
    - c. Change back the attraction strength to 0.02. Introduce repellent walls by writing the following in the loop: `agent.boundary = "bounce_back"`. Is the system robust enough to handle such perturbations? Try different alignment strengths of 1.75 and 5. What is the difference? Try different angular noise values as well.

## Task 4.: Heterogeneity
No that you know how to change the flocking parameters of the agents we can introduce heterogeneity. 

Compared to fully idealized model systems, natural groups are heterogenous in terms of some property. Let's suppose in our flock some of the individuals are faster than others.

1. Change the below code snippet to model a swarm of 20 agents from which half is 10% faster than the other half? You can control the agents' speed with their `v_max` attribute. (**Hint:** You can change the color of the fast agents by setting their `orig_color` attribute to any RGB tuple `(R, G, B)` where R, G, B are integers between 0 and 255).
2. Do you see any effect of such a heterogeneity?
3. What happens when the fast agents are twice as fast as the slow agents (e.g. `v_max=2` and `v_max=1` respectively)? What do you see?
4. Heterogeneity in natural systems are usually on a spectrum and the difference is not binary (fast/slow). Set the maximum velocity of agents in the group as a random uniform distribution between 1 and 2.5. Set the color of the agents in a way that they give useful information about their maximum speed.
5. You can use 30 agents and set the arena to 700x700. Set the boundary condition of the agents to `"bounce_back"` so that they can not cross walls.
6. What do you observe?
7. Is the obstacle avoidance (strong repulsion force) always successful in large groups? Pygame provides useful implementation of [collision groups](https://stackoverflow.com/questions/29640685/how-do-i-detect-collision-in-pygame). Turn on pygame-based obstacle avoidance by adding `physical_obstacle_avoidance=True` in the argment when creating your `Simulation` class instance. Did anything change in the effect you have observed in 4.6.?